In [10]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torchvision import datasets, transforms
from sklearn.metrics import precision_score, recall_score
from torch.utils.data import DataLoader

In [2]:
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()

        self.fc1 = nn.Linear(28*28, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.fc3(x)

        return self.softmax(x)

In [3]:
def compute_metrics(targets, outputs):
    outputs = np.argmax(outputs, axis=1)
    targets = np.array(targets)
    precision = precision_score(targets, outputs, average="weighted")
    recall = recall_score(targets, outputs, average="weighted")

    return precision, recall

In [4]:
def create_dataset(batch_size: int = 64):
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.5,), (0.5,))])

    train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
    test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    return train_loader, test_loader

In [30]:
def train(train_loader, test_loader, optimizer = None, criterion = None, num_epochs = 10, verbose: bool = True):
    model = Perceptron()

    if optimizer is None:
        optimizer = optim.Adam(model.parameters(), lr=0.001)
    if criterion is None:
        criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        model.train().to("cuda")
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data.to("cuda"))
            loss = criterion(output, target.to("cuda"))
            loss.backward()
            optimizer.step()

        model.eval().to("cuda")
        test_loss = 0
        correct = 0
        targets = []
        outputs = []

        with torch.no_grad():
            for data, target in test_loader:
                output = model(data.to("cuda"))
                test_loss += criterion(output, target.to("cuda")).item()
                pred = output.argmax(dim=1, keepdim=True).detach().cpu()
                correct += pred.eq(target.view_as(pred)).sum().item()
                targets.extend(target.detach().cpu().tolist())
                outputs.extend(output.detach().cpu().tolist())

        test_loss /= len(test_loader.dataset)
        accuracy = 100. * correct / len(test_loader.dataset)

        precision, recall = compute_metrics(targets, outputs)

        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%")
        print(f"Precision: {precision:.4f}, Recall: {recall:.4f}")

    torch.save(model.state_dict(), 'perceptron.pth')

In [20]:
train_loader, test_loader = create_dataset()

In [74]:
train(train_loader, test_loader, num_epochs=10)

Epoch 1/10:

Test Loss: 0.0236, Accuracy: 95.92%

Precision: 0.9603, Recall: 0.9592

Epoch 2/10:

Test Loss: 0.0234, Accuracy: 97.01%

Precision: 0.9703, Recall: 0.9701

Epoch 3/10:

Test Loss: 0.0234, Accuracy: 97.15%

Precision: 0.9720, Recall: 0.9715

Epoch 4/10:

Test Loss: 0.0233, Accuracy: 97.76%

Precision: 0.9776, Recall: 0.9776

Epoch 5/10:

Test Loss: 0.0233, Accuracy: 97.68%

Precision: 0.9769, Recall: 0.9768

Epoch 6/10:

Test Loss: 0.0233, Accuracy: 97.59%

Precision: 0.9759, Recall: 0.9759

Epoch 7/10:

Test Loss: 0.0233, Accuracy: 97.86%

Precision: 0.9787, Recall: 0.9786

Epoch 8/10:

Test Loss: 0.0233, Accuracy: 97.96%

Precision: 0.9797, Recall: 0.9796

Epoch 9/10:

Test Loss: 0.0233, Accuracy: 98.10%

Precision: 0.9812, Recall: 0.9810

Epoch 10/10:

Test Loss: 0.0232, Accuracy: 98.02%

Precision: 0.9803, Recall: 0.9802


1. Використайте різну кількість нейронів на вхідному шарі:

Перевизначемо модель

400 нейронів у вхідному шарі (використовуємо лише частину вхідного забраження, відкинувши частину вектора фіч зліва та справа)

In [96]:
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()

        self.fc1 = nn.Linear(400, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(-1, 28*28)
        slice_start = (28*28 - 400) // 2
        slice_end = slice_start + 400
        x = x[:, slice_start: slice_end]
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.fc3(x)

        return self.softmax(x)

In [97]:
train(train_loader, test_loader, num_epochs=10)

Epoch 1/10:

Test Loss: 0.0238, Accuracy: 94.93%

Precision: 0.9502, Recall: 0.9493

Epoch 2/10:

Test Loss: 0.0236, Accuracy: 96.19%

Precision: 0.9621, Recall: 0.9619

Epoch 3/10:

Test Loss: 0.0235, Accuracy: 96.61%

Precision: 0.9662, Recall: 0.9661

Epoch 4/10:

Test Loss: 0.0235, Accuracy: 96.53%

Precision: 0.9656, Recall: 0.9653

Epoch 5/10:

Test Loss: 0.0234, Accuracy: 96.93%

Precision: 0.9694, Recall: 0.9693

Epoch 6/10:

Test Loss: 0.0234, Accuracy: 96.87%

Precision: 0.9688, Recall: 0.9687

Epoch 7/10:

Test Loss: 0.0234, Accuracy: 96.99%

Precision: 0.9699, Recall: 0.9699

Epoch 8/10:

Test Loss: 0.0234, Accuracy: 97.15%

Precision: 0.9715, Recall: 0.9715

Epoch 9/10:

Test Loss: 0.0234, Accuracy: 97.13%

Precision: 0.9714, Recall: 0.9713

Epoch 10/10:

Test Loss: 0.0234, Accuracy: 97.32%

Precision: 0.9734, Recall: 0.9732


Метрики впали, через те, що модель втратила частину інформації з вхідних даних. Сенсу у зменшенні вхідного шару у нашому випадку небагато. З позитивних моментів можна виділити лише швидше навчання моделі.

1200 нейронів у вхідному шарі

In [81]:
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()

        self.fc1 = nn.Linear(1200, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.nn.functional.pad(x, (0, 1200 - 28*28), mode='constant', value=0)
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.fc3(x)

        return self.softmax(x)

In [82]:
train(train_loader, test_loader, num_epochs=10)

Epoch 1/10:

Test Loss: 0.0235, Accuracy: 96.62%

Precision: 0.9664, Recall: 0.9662

Epoch 2/10:

Test Loss: 0.0234, Accuracy: 97.13%

Precision: 0.9714, Recall: 0.9713

Epoch 3/10:

Test Loss: 0.0234, Accuracy: 97.48%

Precision: 0.9749, Recall: 0.9748

Epoch 4/10:

Test Loss: 0.0234, Accuracy: 97.19%

Precision: 0.9721, Recall: 0.9719

Epoch 5/10:

Test Loss: 0.0233, Accuracy: 97.69%

Precision: 0.9770, Recall: 0.9769

Epoch 6/10:

Test Loss: 0.0233, Accuracy: 97.96%

Precision: 0.9797, Recall: 0.9796

Epoch 7/10:

Test Loss: 0.0233, Accuracy: 97.75%

Precision: 0.9775, Recall: 0.9775

Epoch 8/10:

Test Loss: 0.0233, Accuracy: 97.52%

Precision: 0.9753, Recall: 0.9752

Epoch 9/10:

Test Loss: 0.0233, Accuracy: 97.88%

Precision: 0.9789, Recall: 0.9788

Epoch 10/10:

Test Loss: 0.0232, Accuracy: 98.22%

Precision: 0.9822, Recall: 0.9822


метрики аналогічні, як і очікувалося. сенсу брати розмір вхідного шару іншим, ніж кількість пікселів на зображенні у даному випадку не раціонально.

2. Додайте в нейронну мережу прихований шар з різною кількістю нейронів



Збільшимо кілььість параметрів моделі за рахунок збільшення кількості нейронів у проміжних шарах

In [33]:
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()

        self.fc1 = nn.Linear(28*28, 28*28)
        self.bn1 = nn.BatchNorm1d(28*28)
        self.fc2 = nn.Linear(28*28, 28*28)
        self.bn2 = nn.BatchNorm1d(28*28)
        self.fc3 = nn.Linear(28*28, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.fc3(x)

        return self.softmax(x)

In [34]:
train(train_loader, test_loader, num_epochs=10)

Epoch 1/10:
Test Loss: 0.0235, Accuracy: 96.37%
Precision: 0.9639, Recall: 0.9637
Epoch 2/10:
Test Loss: 0.0235, Accuracy: 96.76%
Precision: 0.9679, Recall: 0.9676
Epoch 3/10:
Test Loss: 0.0234, Accuracy: 97.19%
Precision: 0.9721, Recall: 0.9719
Epoch 4/10:
Test Loss: 0.0234, Accuracy: 97.29%
Precision: 0.9730, Recall: 0.9729
Epoch 5/10:
Test Loss: 0.0233, Accuracy: 97.42%
Precision: 0.9742, Recall: 0.9742
Epoch 6/10:
Test Loss: 0.0233, Accuracy: 97.65%
Precision: 0.9766, Recall: 0.9765
Epoch 7/10:
Test Loss: 0.0233, Accuracy: 97.56%
Precision: 0.9759, Recall: 0.9756
Epoch 8/10:
Test Loss: 0.0233, Accuracy: 97.69%
Precision: 0.9769, Recall: 0.9769
Epoch 9/10:
Test Loss: 0.0233, Accuracy: 97.77%
Precision: 0.9778, Recall: 0.9777
Epoch 10/10:
Test Loss: 0.0233, Accuracy: 97.88%
Precision: 0.9790, Recall: 0.9788


метрики погіршилися, це свідчиться про перенавчання моделі

Зменшимо кількість нейронів у проміжних шарах

In [6]:
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()

        self.fc1 = nn.Linear(28*28, 400)
        self.bn1 = nn.BatchNorm1d(400)
        self.fc2 = nn.Linear(400, 200)
        self.bn2 = nn.BatchNorm1d(200)
        self.fc3 = nn.Linear(200, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.fc3(x)

        return self.softmax(x)

In [11]:
train(train_loader, test_loader, num_epochs=10)

Epoch 1/10:
Test Loss: 0.0236, Accuracy: 96.44%
Precision: 0.9646, Recall: 0.9644
Epoch 2/10:
Test Loss: 0.0235, Accuracy: 96.75%
Precision: 0.9678, Recall: 0.9675
Epoch 3/10:
Test Loss: 0.0234, Accuracy: 97.33%
Precision: 0.9734, Recall: 0.9733
Epoch 4/10:
Test Loss: 0.0233, Accuracy: 97.62%
Precision: 0.9763, Recall: 0.9762
Epoch 5/10:
Test Loss: 0.0233, Accuracy: 97.82%
Precision: 0.9784, Recall: 0.9782
Epoch 6/10:
Test Loss: 0.0233, Accuracy: 97.81%
Precision: 0.9782, Recall: 0.9781
Epoch 7/10:
Test Loss: 0.0233, Accuracy: 97.82%
Precision: 0.9782, Recall: 0.9782
Epoch 8/10:
Test Loss: 0.0233, Accuracy: 98.15%
Precision: 0.9815, Recall: 0.9815
Epoch 9/10:
Test Loss: 0.0232, Accuracy: 98.17%
Precision: 0.9817, Recall: 0.9817
Epoch 10/10:
Test Loss: 0.0232, Accuracy: 98.13%
Precision: 0.9814, Recall: 0.9813


Метрики залишилася на приблизно такому ж рівні (зменшилися на 0.08%), це значить що попередня модель можливо мала більшу складність ніж потрібно

Зменшимо її розмір ще більше

In [13]:
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()

        self.fc1 = nn.Linear(28*28, 200)
        self.bn1 = nn.BatchNorm1d(200)
        self.fc2 = nn.Linear(200, 100)
        self.bn2 = nn.BatchNorm1d(100)
        self.fc3 = nn.Linear(100, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.fc3(x)

        return self.softmax(x)

In [31]:
train(train_loader, test_loader, num_epochs=10)

Epoch 1/10:
Test Loss: 0.0236, Accuracy: 96.48%
Precision: 0.9650, Recall: 0.9648
Epoch 2/10:
Test Loss: 0.0234, Accuracy: 97.00%
Precision: 0.9701, Recall: 0.9700
Epoch 3/10:
Test Loss: 0.0233, Accuracy: 97.63%
Precision: 0.9764, Recall: 0.9763
Epoch 4/10:
Test Loss: 0.0233, Accuracy: 97.61%
Precision: 0.9764, Recall: 0.9761
Epoch 5/10:
Test Loss: 0.0234, Accuracy: 97.49%
Precision: 0.9750, Recall: 0.9749
Epoch 6/10:
Test Loss: 0.0233, Accuracy: 97.90%
Precision: 0.9790, Recall: 0.9790
Epoch 7/10:
Test Loss: 0.0233, Accuracy: 97.92%
Precision: 0.9793, Recall: 0.9792
Epoch 8/10:
Test Loss: 0.0233, Accuracy: 98.00%
Precision: 0.9800, Recall: 0.9800
Epoch 9/10:
Test Loss: 0.0233, Accuracy: 97.93%
Precision: 0.9793, Recall: 0.9793
Epoch 10/10:
Test Loss: 0.0233, Accuracy: 98.07%
Precision: 0.9807, Recall: 0.9807


Бачимо, що зменшення кількості параметрів все-таки почало погіршувати метрики, варто повернутися до попередньої кількості парметрів. 

3. Додайте кілька прихованих шарів в мережу з різною кількістю нейронів в кожному шарі. Яким чином це впливає на якість навчання нейронної мережі.


додамо ще один прихований шар, але спробуємо залишити загальну кількість параметрів моделі на тому ж рівні

In [37]:
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()

        self.fc1 = nn.Linear(28*28, 200)
        self.bn1 = nn.BatchNorm1d(200)
        self.fc2 = nn.Linear(200, 200)
        self.bn2 = nn.BatchNorm1d(200)
        self.fc2 = nn.Linear(200, 100)
        self.bn2 = nn.BatchNorm1d(100)
        self.fc3 = nn.Linear(100, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.fc3(x)

        return self.softmax(x)

In [38]:
train(train_loader, test_loader, num_epochs=10)

Epoch 1/10:
Test Loss: 0.0236, Accuracy: 96.56%
Precision: 0.9658, Recall: 0.9656
Epoch 2/10:
Test Loss: 0.0234, Accuracy: 97.10%
Precision: 0.9710, Recall: 0.9710
Epoch 3/10:
Test Loss: 0.0234, Accuracy: 97.19%
Precision: 0.9722, Recall: 0.9719
Epoch 4/10:
Test Loss: 0.0234, Accuracy: 97.50%
Precision: 0.9752, Recall: 0.9750
Epoch 5/10:
Test Loss: 0.0234, Accuracy: 97.28%
Precision: 0.9731, Recall: 0.9728
Epoch 6/10:
Test Loss: 0.0233, Accuracy: 97.80%
Precision: 0.9780, Recall: 0.9780
Epoch 7/10:
Test Loss: 0.0233, Accuracy: 97.84%
Precision: 0.9786, Recall: 0.9784
Epoch 8/10:
Test Loss: 0.0233, Accuracy: 97.87%
Precision: 0.9788, Recall: 0.9787
Epoch 9/10:
Test Loss: 0.0233, Accuracy: 97.92%
Precision: 0.9792, Recall: 0.9792
Epoch 10/10:
Test Loss: 0.0232, Accuracy: 98.08%
Precision: 0.9809, Recall: 0.9808


Підвищення якості моделі немає 

4.Використовуйте різну кількість епох

In [39]:
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()

        self.fc1 = nn.Linear(28*28, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.fc3(x)

        return self.softmax(x)

In [40]:
train(train_loader, test_loader, num_epochs=30)

Epoch 1/30:
Test Loss: 0.0237, Accuracy: 95.71%
Precision: 0.9580, Recall: 0.9571
Epoch 2/30:
Test Loss: 0.0234, Accuracy: 97.02%
Precision: 0.9703, Recall: 0.9702
Epoch 3/30:
Test Loss: 0.0234, Accuracy: 97.46%
Precision: 0.9749, Recall: 0.9746
Epoch 4/30:
Test Loss: 0.0234, Accuracy: 97.49%
Precision: 0.9750, Recall: 0.9749
Epoch 5/30:
Test Loss: 0.0233, Accuracy: 97.64%
Precision: 0.9765, Recall: 0.9764
Epoch 6/30:
Test Loss: 0.0233, Accuracy: 97.88%
Precision: 0.9789, Recall: 0.9788
Epoch 7/30:
Test Loss: 0.0233, Accuracy: 97.77%
Precision: 0.9779, Recall: 0.9777
Epoch 8/30:
Test Loss: 0.0233, Accuracy: 97.76%
Precision: 0.9777, Recall: 0.9776
Epoch 9/30:
Test Loss: 0.0233, Accuracy: 97.98%
Precision: 0.9798, Recall: 0.9798
Epoch 10/30:
Test Loss: 0.0233, Accuracy: 97.82%
Precision: 0.9783, Recall: 0.9782
Epoch 11/30:
Test Loss: 0.0233, Accuracy: 97.88%
Precision: 0.9788, Recall: 0.9788
Epoch 12/30:
Test Loss: 0.0233, Accuracy: 97.99%
Precision: 0.9801, Recall: 0.9799
Epoch 13/30:


Бачимо, що довше тренування має сенс, але тренування довше 15 епох не має сенсу

5. Використовуйте різні розміри міні-вибірки (batch_size)

In [44]:
train_loader, test_loader = create_dataset(batch_size=32)
train(train_loader, test_loader, num_epochs=15)

Epoch 1/15:
Test Loss: 0.0471, Accuracy: 96.03%
Precision: 0.9607, Recall: 0.9603
Epoch 2/15:
Test Loss: 0.0468, Accuracy: 96.78%
Precision: 0.9680, Recall: 0.9678
Epoch 3/15:
Test Loss: 0.0466, Accuracy: 97.18%
Precision: 0.9719, Recall: 0.9718
Epoch 4/15:
Test Loss: 0.0465, Accuracy: 97.48%
Precision: 0.9748, Recall: 0.9748
Epoch 5/15:
Test Loss: 0.0465, Accuracy: 97.59%
Precision: 0.9759, Recall: 0.9759
Epoch 6/15:
Test Loss: 0.0464, Accuracy: 97.71%
Precision: 0.9772, Recall: 0.9771
Epoch 7/15:
Test Loss: 0.0464, Accuracy: 97.84%
Precision: 0.9784, Recall: 0.9784
Epoch 8/15:
Test Loss: 0.0464, Accuracy: 97.76%
Precision: 0.9777, Recall: 0.9776
Epoch 9/15:
Test Loss: 0.0464, Accuracy: 98.05%
Precision: 0.9805, Recall: 0.9805
Epoch 10/15:
Test Loss: 0.0463, Accuracy: 98.07%
Precision: 0.9807, Recall: 0.9807
Epoch 11/15:
Test Loss: 0.0464, Accuracy: 97.96%
Precision: 0.9797, Recall: 0.9796
Epoch 12/15:
Test Loss: 0.0464, Accuracy: 97.99%
Precision: 0.9800, Recall: 0.9799
Epoch 13/15:


покращення не видно, зменшення розміру батча призвело до меншої загальності моделі

In [45]:
train_loader, test_loader = create_dataset(batch_size=128)
train(train_loader, test_loader, num_epochs=15)

Epoch 1/15:
Test Loss: 0.0120, Accuracy: 94.46%
Precision: 0.9486, Recall: 0.9446
Epoch 2/15:
Test Loss: 0.0118, Accuracy: 97.21%
Precision: 0.9722, Recall: 0.9721
Epoch 3/15:
Test Loss: 0.0118, Accuracy: 97.36%
Precision: 0.9736, Recall: 0.9736
Epoch 4/15:
Test Loss: 0.0117, Accuracy: 97.73%
Precision: 0.9774, Recall: 0.9773
Epoch 5/15:
Test Loss: 0.0117, Accuracy: 97.76%
Precision: 0.9776, Recall: 0.9776
Epoch 6/15:
Test Loss: 0.0117, Accuracy: 97.96%
Precision: 0.9796, Recall: 0.9796
Epoch 7/15:
Test Loss: 0.0117, Accuracy: 97.88%
Precision: 0.9789, Recall: 0.9788
Epoch 8/15:
Test Loss: 0.0117, Accuracy: 97.96%
Precision: 0.9797, Recall: 0.9796
Epoch 9/15:
Test Loss: 0.0117, Accuracy: 97.83%
Precision: 0.9784, Recall: 0.9783
Epoch 10/15:
Test Loss: 0.0117, Accuracy: 98.12%
Precision: 0.9813, Recall: 0.9812
Epoch 11/15:
Test Loss: 0.0117, Accuracy: 97.98%
Precision: 0.9799, Recall: 0.9798
Epoch 12/15:
Test Loss: 0.0117, Accuracy: 98.17%
Precision: 0.9818, Recall: 0.9817
Epoch 13/15:


а збільшення батч сайзу трішки теж не покращило метрики.